# Explore NEO data with [Glue](http://docs.glueviz.org/en/stable/index.html)

*Glue allows users to build linked, interactive figures from files and python datasets.  It can be run as a standalone application or launched from python.  In this tutorial, we will explore running glue from a jupyter notebook.*

*We'll work with [Near-Earth object (NEO)](https://en.wikipedia.org/wiki/Near-Earth_object) data. I downloaded NEO data from the [JPL Small-Body Database](https://ssd.jpl.nasa.gov/sbdb_query.cgi) and the [NEO Earth Close Approaches archive](https://cneos.jpl.nasa.gov/ca/).  Then I cleaned up the data a bit, [see here](https://github.com/ageller/IntroToGlue/blob/main/data/prepNEOdata.ipynb).*

<img src="https://upload.wikimedia.org/wikipedia/commons/c/ce/Asteroids-KnownNearEarthObjects-Animation-UpTo20180101.gif" width="100%" align="center">

*animated gif from [here](https://commons.wikimedia.org/wiki/File:Asteroids-KnownNearEarthObjects-Animation-UpTo20180101.gif)*


In [1]:
%gui qt5

In [2]:
#import necessary libraries
import pandas as pd
from glue import qglue

# Read in the data

## [JPL Small-Body Database](https://ssd.jpl.nasa.gov/sbdb_query.cgi)

In [3]:
#the low_memory=False flag suppresses a warning message for a few columns with mixed data types 
jplsbdb = pd.read_csv('data/sbdb_query_results.csv', low_memory=False) 
jplsbdb.iloc[:3]

,id,spkid,full_name,pdes,name,prefix,neo,pha,H,G,...,n_obs_used,n_del_obs_used,n_dop_obs_used,condition_code,rms,two_body,A1,A2,A3,DT
0,a0000433,2000433,433 Eros (A898 PA),433,Eros,NaN,Y,N,10.43,0.46,...,9130,4.0,2.0,0.0,0.29796,NaN,NaN,NaN,NaN,NaN
1,a0000719,2000719,719 Albert (A911 TB),719,Albert,NaN,Y,N,15.51,NaN,...,1894,NaN,NaN,0.0,0.39775,NaN,NaN,NaN,NaN,NaN
2,a0000887,2000887,887 Alinda (A918 AA),887,Alinda,NaN,Y,N,13.87,-0.12,...,2624,NaN,NaN,0.0,0.39776,NaN,NaN,NaN,NaN,NaN


## [NEO Earth Close Approaches archive](https://cneos.jpl.nasa.gov/ca/)

(I cleaned the data file so that it can be used more easily; see the data/prepNEOdata.ipynb.)

In [4]:
neoca = pd.read_csv('data/cneos_closeapproach_data-cleaned.csv')
neoca.iloc[:3]

,pdes,Object,Close-Approach (CA) Date,CA Distance Nominal (AU),CA Distance Minimum (AU),V relative (km/s),V infinity (km/s),H (mag),Diameter (km),extra
0,509352,509352 (2007 AG),1900.096844,0.00963,0.00963,8.69,8.65,20.1,410000.0,a0509352
1,2014 SC324,(2014 SC324),1900.113578,0.03997,0.03997,10.65,10.65,24.3,59500.0,bK14SW4C
2,2012 UK171,(2012 UK171),1900.118827,0.04982,0.04982,7.16,7.15,24.4,55500.0,bK12UH1K


# Start Glue with these data

*[qglue](http://docs.glueviz.org/en/latest/python_guide/glue_from_python.html) is a way to send python data structures (Numpy arrays, Pandas dataframes, Astropy tables, others) to glue. It returns an application object wich contains lots of state about the application.*


*The following code is supposed to allow glue to run within a notebook without blocking.  [See here.](http://docs.glueviz.org/en/stable/python_guide/glue_from_python.html#using-qglue-with-the-ipython-jupyter-notebook)*

<br>
<div style='background-color:#eeffcc; padding:10px; border: 1px solid #e1e4e5'>
%gui qt
</div>
<br>
<div style='background-color:#eeffcc; padding:10px; border: 1px solid #e1e4e5'>
app = qglue(jplsbdb = jplsbdb, neoca = neoca)
</div>
<br>

*But this does not work on my end. I recommend that you try it to see if you can make it work because it may simplify the workflow.  For now, I will run glue as a backgroundjob...*

In [ ]:
from IPython.lib import backgroundjobs as bg

In [ ]:
def runglue(*args):
    app = qglue(jplsbdb = args[0], neoca = args[1])
    return app

In [ ]:
jobs = bg.BackgroundJobManager()
jobs.new(runglue, jplsbdb, neoca)

In [ ]:
jobs.status()

In [ ]:
jobs[0].result

## Manipulate the data in glue

*Make a few plots and selections.  Let's try to reproduce (approximately) the image below.  If you're running as a background job, you will have to (save and) quite glue in order to have access to the data products in the notebook.  If you were able to run with the notebook magic command, you should be able to leave your glue session live.*

## Next, lets explore the selected data in python

jobs[0].result *holds the return value from runglue, which is the qglue app.  Within that app you have access to the "data_collection".  Data Collections are list-like and contain each dataset from Glue. Individual datasets in Glue are dictionary-like.*

In [ ]:
jobs[0].result.data_collection

In [ ]:
from glue.app.qt.application import GlueApplication

In [ ]:
from glue.core import DataCollection

In [4]:
%gui qt

In [ ]:
dc = DataCollection()
dc['jplsbdb'] = jplsbdb

In [ ]:
app = GlueApplication(dc)
app.start()

In [ ]:
app = qglue(jplsbdb = jplsbdb, neoca = neoca)